# Automatic Agent Optimization with MIPRO

MIPRO (Multiprompt Instruction PRoposal Optimizer) is an optimizer you can use to create the best prompts from your task. You can read the original paper [here](https://arxiv.org/abs/2406.11695).

This optimizer was popularized by the [DSPy library](https://dspy.ai/) that provided the first and most commonly used implementation.

In the notebook below, we will re-implement the algorithm from scratch to get a better understanding of how it works.

## Optimization strategy

The MIPRO algorithm was developed to optimize the multi-step LLM applications when you don't have labels for each step but rather global label for the entire task. This makes it a great algorithm to automatically optimize agents !

## Define the agent

To test the optimization algorithm, we are going to start by creating a very simple agent.

In [4]:
import os
import getpass

%pip install openai

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY")


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
from openai import OpenAI
import re


class CalculatorTool:
    def __init__(self, name="calculator"):
        self.name = name

    def __call__(self, operation: str) -> float:
        return eval(operation)


class SummarizeTool:
    def __init__(self, name: str = "summarization", summarization_prompt: str = ""):
        self.name = name
        self.openai_client = OpenAI()
        self.summarization_prompt = summarization_prompt

    def __call__(self, text: str) -> str:
        completion = self.openai_client.chat.completions.create(
            messages=[
                {"role": "user", "content": self.summarization_prompt.format(text=text)}
            ],
            model="gpt-4o",
        )

        return completion.choices[0].message.content


class Agent:
    def __init__(self, client: OpenAI, system: str = "", tools: list = []) -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        self.tools = tools

        if self.system:
            self.messages.append({"role": "system", "content": system})

    def send_message(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})

        completion = self.client.chat.completions.create(
            model="gpt-4o", messages=self.messages
        )

        result = completion.choices[0].message.content
        self.messages.append({"role": "assistant", "content": result})
        return result

    def call_agent(self, message: str = "", max_iterations: int = 10) -> str:
        i = 0
        next_prompt = message
        while i < max_iterations:
            i += 1
            result = self.send_message(next_prompt)

            if "STOP" in result:
                break
            elif "Action" in result:
                action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
                chosen_tool = action[0][0]
                arg = action[0][1]

                tool_names = [x.name for x in self.tools]
                if chosen_tool in tool_names:
                    tool = [x for x in self.tools if x.name == chosen_tool][0]

                    result_tool = tool(arg)
                    next_prompt = f"Result: {result_tool}"
                    # self.messages.append({"role": "assistant", "content": next_prompt})
                else:
                    raise ValueError(f"Could not find tool {chosen_tool}")

        return self.messages


system_prompt = """
You are an LLM agent that can both summarize and calculate.

You have access to the tools:
- calculator: Calculate a formula
- summarize: Summarize some text

To call a tool, return the text:
Action: <tool_name>: <tool_parameter>

Return STOP when finished
"""

summarize_prompt = "Summarize the text provided below."
agent = Agent(
    OpenAI(), system_prompt, [CalculatorTool(), SummarizeTool(summarize_prompt)]
)

questions = [
    "What is 2 + 3 * 1031 ?",
    "Write a long poem and then summarize it in one sentence",
]

print("----- Questions chatbot ------")

for question in questions:
    print(question + "\n")
    res = agent.call_agent(question)
    print("   -> " + res[-1]["content"])

----- Questions chatbot ------
What is 2 + 3 * 1031 ?

   -> Now, add 2 to the result of the multiplication:

2 + 3093 = 3095

STOP
Write a long poem and then summarize it in once sentence



ValueError: Could not find tool summarize